In [5]:
%matplotlib qt
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.insert(0, "../scripts")
import util
import camera

# from sklearn.cluster import DBSCAN

np.set_printoptions(precision=4)
!pwd

/home/zpyang/git/visnet/src/visnet/notebooks


In [2]:
%clear
import camera
cam_param = [642.0926, 642.0926, 1000.5, 1000.5,0]
# [x y z roll pitch yaw]
cam_poses = np.array([
    [20, 20, 12, 0, 0, -2.2],
    [20, -20, 12, 0, 0, 2.2],
    [-20, -20, 12, 0, 0, 0.7],
    [-20, 20, 12, 0, 0, -0.7],
])

cam_group = camera.CamGroup(cam_param, cam_poses[0:3,:])
msmts = cam_group.get_group_measurement(np.array([[0,0,10], [10,10,10]]))[0]

def msmt_association(measurements, hypothesis):
    """
    INPUTS:
    measurements: From the m-th camera, shape: (n_measurements, 2+1)
    hypothesis: State hypothesis on cameras at t-1 of all tracked objects, shape: (n_targets, 2+1)

    RETURN:
    msmt_m: Associated measurements of m-th camera, order in the same way as the targets. |msmt_m[i] <-> tracks[i]|
    """
    # assert(type(prior_states)==np.ndarray)

    msmt_m = []
    for hi in hypothesis:
        w = util.squared_error(measurements, hi)
        argmax_w = np.argmax(w)
        msmt_m.append(measurements[argmax_w])
    
    return np.array(msmt_m)

def observe_fn(cam_group, particles):
        """
        Parameters:
            @ particles : all particles at a step
        Returns:
            @ hypothesis from x using the camera projection matrix
            This returns list of arrays which corresponds to hypothesis of particles on each camera
            |hypo[i] <-> camera[i]|
        """

        pos = particles[:, 0:3]
        hypothesis = cam_group.get_group_measurement(pos)

        return hypothesis

def prior_fn(n_particles, spawn_regions=None):
        if spawn_regions is not None:
            n_spawn = spawn_regions.shape[0]
            for region in spawn_regions:
                pos = np.random.normal(region, 2*np.ones(3), (n_particles, 3))
                vel = np.random.normal(np.zeros(3), np.ones(1), (n_particles, 3))
                # label = np.random.randint(0, 2, (n,1))
                particles = np.hstack([pos, vel])
            return particles
        else:
            pos = np.random.uniform(low=[-20, -20, 5], high=[20, 20, 25], size=(n_particles,3))
            vel = np.random.uniform(low=np.zeros(3), high=np.ones(3), size=(n_particles,3))

            # label = np.random.randint(0, 2, (n,1))
            particles = np.hstack([pos, vel])
            return particles

def weight_fn(self, msmt, hypo, sigma, verbose=False):
        """
        Iterate through each camera, update weights
        @ msmt: [{yj}_0, {yj}_1, ...{yj}_n]    n cameras
        @ hypo: [bearing_0, bearing_1, ....bearing_n]   n_cameras
        """

        n_particles = hypo[0].shape[0]
        weights = np.ones(n_particles)

        for zi, hi in zip(msmt, hypo):
            if zi[0] < 0: # got no measurements so weights are unchanged
                wi = np.ones(n_particles)
            wi = util.squared_error(hi, zi, sigma=sigma)
            weights *= wi

        return weights / np.sum(weights)

n_particles = 10
particles = prior_fn(n_particles)

hypothesis = observe_fn(cam_group, particles)

In [3]:
def target_traj_circle(t):
    x = 15*np.sin(2 * np.pi * 0.01 * t / 3)
    y = 10*np.cos(2 * np.pi * 0.01 * t / 3)
    z = 20
    return np.array([x,y,z])

def target_traj_straight(t, start, finish, tf):
    max_dist = np.linalg.norm(start-finish)
    v_max = (finish-start) / tf
    # x = -15 + t * 0.1
    # y = 0
    # z = 20
    return start + v_max * t

def target_stationary(start):
    return start

In [4]:
class Track:
    """
    This is a data structure for representing tracked targets
    """
    def __init__(self, n_particles: int, spawn):
        self.n_particles = n_particles
        self.spawn_region = spawn_region
        self.map_state = None # MAP estimate, particle with the max weight
        self.mean = None
        self.cov = None
        self.weights = np.ones(n_particles)
        self.particles = None

        self.cam_group = None
        self.hypothesis = None
        self.map_hypo = None # MAP estimate of pixel locations on different cameras
    
    def update(self, weights):
        """
        Update track after particle filter update
        """
        self.weight_sum = np.sum(weights)
        self.weights = weights / self.weight_sum
        
        # mean and cov
        self.mean_hypothesis = np.sum(self.hypothesis.T * self.weights, axis=-1).T
        self.mean_state = np.sum(self.particles.T * self.weights, axis=-1).T
        self.cov_state = np.cov(self.particles, rowvar=False, aweights=self.weights)
        
        # Maximum-a-posterior MAP
        argmax_weight = np.argmax(self.weights)
        self.map_state = self.particles[argmax_weight]
        self.map_hypothesis = self.hypothesis[argmax_weight]
        
        self.n_eff = (1.0 / np.sum(self.weights ** 2)) / self.n_particles
        pass

In [ ]:
class ParticleFilter():
    """
    unknown number of targets
    """
    def __init__(self,
                 n_eff_thresh=1,
                 sigma=150,
                 resample_proportion=None,
                 verbose=False,
                ):
        self.n_eff_thresh = n_eff_thresh
        self.resample_proportion = resample_proportion or 0.0
        self.verbose = verbose
        self.tracks = []

    def update(self, track: Track, observation=None):
        particles = self.dynamics_d(track.particles)
        
        hypothesis = self.observe_fn(track.cam_group, particles)
        hypothesis = hypothesis.reshape(track.n_particles, track.cam_group.n_cams * 2)

        weights = self.weight_fn(observation, hypothesis)

        return weights

        # n_eff = (1.0 / np.sum(weights ** 2)) / track.n_particles
        # self.weight_entropy = np.sum(self.weights * np.log(self.weights, where=self.weights!=0))|

    def resample(self, n_eff, particles, weights, resample_fn):
        if n_eff < self.n_eff_thresh:
            indices = resample_fn(self.weights)
            particles = particles[indices, :]
            weights = np.ones(self.n_particles) / self.n_particles
        
        return particles, weights
        
    dt = 1/10
    A = np.block([
        [np.eye(3), dt*np.eye(3)],
        [np.zeros((3,3)), np.eye(3)]
    ])
    # A = np.block([
    #     [A, np.zeros((6,1))],
    #     [np.zeros((1,6)), 1]
    # ])
    @classmethod
    def dynamics_d(self, x):
        """
        Discrete dynamics, last state is the target id
        """
        n, d = x.shape
        w = np.zeros((d, n))
        w[3:6] = np.random.normal(0, 1, (3,n))
        x_1 = ParticleFilter.A @ x.T + w

        return x_1.T

In [ ]:
start = np.array([-18, 0, 15])
end = np.array([20, 0, 15])

n_targets = 2
tracks = []

t_final = 100

verbose = False
spawn_region = start
n_particles = 500

pf = ParticleFilter(
    n_eff_thresh=1,
    sigma=150,
    resample_proportion=0
)

for t in range(t_final):
    target_1 = target_traj_straight(t, start, end, t_final)
    target_2 = target_traj_straight(t, start-[0,5,0], end-[0,5,0], t_final)
    # target_2 = target_traj_circle(t)
    # target_1 = target_stationary(np.array([-5,5,20]))
    # target_2 = target_stationary(np.array([-5,-5,20]))
    targets = np.vstack([target_1, target_2])
    # targets = np.vstack([target_1])

    state.append(targets)
    
    #each camera can give multiple measurements
    z = cam_group.get_group_measurement(targets)

    pf1.update(z)
    pf1.cluster_fn(eps=3, min_samples=10)
    
    particles.append(pf1.particles)
    clusters_list.append(pf1.clusters)

    # state_est.append(pf1.map_state)
    weights.append(pf1.weights)
    # cov.append(pf1.cov_state)
    pf1.resample(util.residual_resample)
    # pf1.resample(util.residual_resample)
    
    n_eff.append(pf1.n_eff)

state = np.array(state)
print(state.shape)
state = state.reshape(t_final, n_targets*3)
n_eff = np.array(n_eff)

particles = np.array(particles)
# state_est = np.array(state_est)
weights = np.array(weights)
# err1 = state[:,0:3] - state_est[:, 0:3]

In [ ]:
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
from matplotlib import cm
# Attaching 3D axis to the figure
fig = plt.figure("Weight visualization and Clustering", figsize=(20, 10))

ax1 = fig.add_subplot(1, 3, 1, projection="3d")
ax2 = fig.add_subplot(1, 3, 2, projection="3d")
ax3 = fig.add_subplot(1, 3, 3, projection="3d")
ax1.view_init(30,0)

def update_frame(i):
    # NOTE: there is no .set_data() for 3 dim data...
    x = particles[i, :, 0]
    y = particles[i, :, 1]
    z = particles[i, :, 2]
    w = weights[i, :]
    
    ax1.cla()
    ax1.set_xlim3d(-25,25)
    ax1.set_ylim3d(-25,25)
    ax1.set_zlim3d(0,25)
    ax1.scatter(x, y, z, cmap=cm.turbo, c=w, s=2)
    ax1.set_xlabel('x, m')
    ax1.set_ylabel('y, m')
    ax1.set_zlabel('z, m')
    
    ax2.cla()
    ax2.set_xlim3d(-25,25)
    ax2.set_ylim3d(-25,25)
    ax2.set_zlim3d(0,25)
    ax2.set_xlabel('x, m')
    ax2.set_ylabel('y, m')
    ax2.set_zlabel('z, m')

    clusters_i = clusters_list[i]
    colors = [plt.cm.tab10(each) for each in np.linspace(0, 1, len(clusters_i))]
    for cluster, col in zip(clusters_i, colors):
        ax2.scatter(
            cluster[:, 0],
            cluster[:, 1],
            cluster[:, 2],
            "o",
            color=col,
        )
        
    ax3.cla()
    ax3.set_xlim3d(-25,25)
    ax3.set_ylim3d(-25,25)
    ax3.set_zlim3d(0,25)
    ax3.set_xlabel('x, m')
    ax3.set_ylabel('y, m')
    ax3.set_zlabel('z, m')
    # print(pf1.tracks[i][0])
    if pf1.tracks[i][0] is not None:
        
        ax3.scatter(pf1.tracks[i][0][0], pf1.tracks[i][0][1], pf1.tracks[i][0][2])
    return None


ani = FuncAnimation(
    fig, update_frame, t_final, fargs=[],
    interval=int(1000 / 10), blit=False)
# ani.save("../videos/particles.mp4")
# HTML(ani.to_html5_video())
# plt.close()

In [ ]:
print(state.shape)
plt.figure(figsize=(10,10))
ax = plt.axes(projection="3d")
ax.plot3D(state[:,0], state[:,1], state[:,2], marker='+', label='truth1', markersize=10)
# ax.plot3D(state[:,3], state[:,4], state[:,5], '+-', label='truth2')
ax.scatter(state_est[:,0], state_est[:,1], state_est[:,2], marker='x', label='est')
ax.set_xlabel('x [m]')
ax.set_ylabel('y [m]')
ax.set_zlabel('z [m]')
ax.set_xlim3d(-25,25)
ax.set_ylim3d(-25,25)
ax.set_zlim3d(0,25)
ax.view_init(60,-90)

plt.legend()

plt.figure(figsize=(10,8))
plt.subplot(3,1,1)
plt.plot(err1)
plt.title('error plot')
plt.xlabel('# of frames')
plt.ylabel('error[m]')

plt.subplot(3,1,2)
plt.plot(n_eff)
plt.title('n_eff')
plt.xlabel('# of frames')
plt.ylabel('n_eff')

plt.subplot(3,1,3)
plt.plot(state_est[:,-1])
plt.title('label')
plt.xlabel('# of frames')
plt.ylabel('label')
plt.show()